In [ ]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import random
import sys
import copy
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import HeteroData
from tqdm import tqdm
import json
from itertools import combinations
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


In [ ]:
!pip install torch

In [ ]:

!pip install torch_geometric

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
path = PATH_FOLDER
os.chdir(path)

In [ ]:
PATH_FOLDER = "/content/drive/MyDrive/Darmoth-Cre-Gene-Project"
PATH_CRE_ATTRIBUTES = "Dataset/LZ_cRE_attributes/LZ_chr1.txt"
PATH_CRE_DNA = "Dataset/cRE_DNA/chr1.npy"
PATH_CRE_EDGE = "Dataset/LZ_cREedge/LZ_t4_chr1.txt"

PATH_GENE_ATTRIBUTES = "Dataset/LZ_GeneTSS300_attributes/TSS300_LZ_chr1.txt"
PATH_GENE_DNA = "Dataset/GENE_DNA/chr1.npy"
PATH_GENE_EDGE = "Dataset/LZ_GENE-cRE_edge/Panc_GCindex_chr1.txt"

PATH_CRE_PREDICTION = 'Result/drepreds.pt'

In [ ]:
#import dataset
cREatt= pd.read_table(PATH_CRE_ATTRIBUTES,header=None,sep=" ")

cREatt1=cREatt.iloc[:,[3,5,6,7,8,9,10,11,12,13,14]]
cREatt1.to_numpy()

cRE_att = torch.tensor(StandardScaler().fit_transform(cREatt1.to_numpy()), dtype=torch.float)

cRE_DNA=np.load(PATH_CRE_DNA)

cRE_edge = pd.read_table(PATH_CRE_EDGE,header=None,sep=" ")

cRE_edgeindex=torch.tensor([cRE_edge[:][0],cRE_edge[:][1]], dtype=torch.long)

cRE_all=torch.cat([cRE_att, torch.tensor(cRE_DNA)], 1)

In [ ]:
print("cRE_edgeindex.shape",cRE_edgeindex.shape)
print("cRE_DNA",cRE_DNA.shape)
print("cRE_al",cRE_all.shape)
print("cRE_att",cRE_att.shape)

cRE_edgeindex.shape torch.Size([2, 79389])
cRE_DNA (9946, 768)
cRE_al torch.Size([9946, 779])
cRE_att torch.Size([9946, 11])


In [ ]:
cRE_prediction=torch.load(PATH_CRE_PREDICTION)


In [ ]:
GENEatt= pd.read_table(PATH_GENE_ATTRIBUTES,header=0,sep=" ")
GENEatt1=GENEatt.iloc[:,[3,4,5,6,7,8,9,10,11,12,13]]
GENE_att = torch.tensor(StandardScaler().fit_transform(GENEatt1.to_numpy()), dtype=torch.float)


GENE_DNA=np.load(PATH_GENE_DNA)
GENE_edge=pd.read_table(PATH_GENE_EDGE,header=None,sep=" ")
# a=np.array(np.hstack((GENE_edge[:][0],GENE_edge[:][1])))
# b=np.array(np.hstack((GENE_edge[:][1],GENE_edge[:][0])))
# GENE_edgeindex=torch.tensor([a,b], dtype=torch.long)
GENE_edgeindex=torch.tensor([GENE_edge[:][0],GENE_edge[:][1]], dtype=torch.long)-1
GENE_all = torch.cat([GENE_att, torch.tensor(GENE_DNA)], 1)

In [ ]:
CGedge=0-torch.ones(cRE_att.size()[0],GENE_att.size()[0])

for i in range(GENE_edgeindex.size()[1]):
    CGedge[GENE_edgeindex[0][i]][GENE_edgeindex[1][i]]=1



In [ ]:
print("GENE_edgeindex.shape",GENE_edgeindex.shape)
print("GENE_DNA",GENE_DNA.shape)
print("GENE_al",GENE_all.shape)
print("GENEatt",GENE_att.shape)
print("CGedge",CGedge.shape)

GENE_edgeindex.shape torch.Size([2, 2718])
GENE_DNA (1984, 768)
GENE_al torch.Size([1984, 779])
GENEatt torch.Size([1984, 11])
CGedge torch.Size([9946, 1984])


In [ ]:

num_neg=0
m=cRE_att.size()[0]-1
avil=torch.zeros(cRE_att.size()[0],GENE_att.size()[0])
maxneg=int(len(GENE_edgeindex[1]))
while(num_neg < maxneg):
    t=random.randint(0,(cRE_att.size()[0]*GENE_att.size()[0])-1)
    a=t//cRE_att.size()[0]
    b=t%GENE_att.size()[0]
    if(CGedge[a][b]==-1 and avil[a][b]==0):
        CGedge[a][b]=0
        num_neg+=1
        avil[a][b]=1
 

In [ ]:
print(num_neg)

2718


In [ ]:
print("CGedge",CGedge.shape)

CGedge torch.Size([9946, 1984])


In [ ]:
listlength=1000
cREsamplesize=100
genesamplesize=100
dataset=[]
# avil=torch.zeros(cRE_att.shape[0])
# avilnum=cRE_att.shape[0]
for i in tqdm(range(listlength)):
    x_dataset=[]
    #x1_dataset=[]
    edge_index=[]
    cREedgelista=[]
    cREedgelistb=[]
    GENEedgelista=[]
    GENEedgelistb=[]
    ycRE=[]
    yGENE=[]
    xcRE=random.sample(range(cRE_all.shape[0]),k=cREsamplesize)
    xgene=random.sample(range(GENE_att.shape[0]),k=genesamplesize)
    for m in range(cREsamplesize):
#         if(avil[xi[m]]==0):
#             avil[xi[m]]=1
#             avilnum-=1
        for n in range(genesamplesize):
            yGENE.append(CGedge[xcRE[m]][xgene[n]])
            if(CGedge[xcRE[m]][xgene[n]]==1):
                    GENEedgelista.append(m)
                    GENEedgelistb.append(n)
        for n in range(cREsamplesize):
            ycRE.append(cRE_prediction[xcRE[m]][xcRE[n]])
            if(cRE_prediction[xcRE[m]][xcRE[n]]==1):
                cREedgelista.append(m)
                cREedgelistb.append(n) 

    hetero = HeteroData()
    hetero['cRE'].x= torch.index_select(cRE_all, 0, torch.tensor(xcRE).long())
    hetero['gene'].x=torch.index_select(GENE_all, 0, torch.tensor(xgene).long())
    hetero['cRE'].y=torch.tensor(ycRE).long()
    hetero['gene'].y=torch.tensor(yGENE).long()
    hetero['cRE','interactswith','cRE'].edge_index=torch.tensor([cREedgelista,cREedgelistb], dtype=torch.long)
    hetero['cRE','regulates','gene'].edge_index=torch.tensor([GENEedgelista,GENEedgelistb], dtype=torch.long)
    dataset.append(hetero)



100%|██████████| 1000/1000 [09:10<00:00,  1.82it/s]


In [ ]:
print(len(dataset))

1000


In [ ]:
    
from sklearn.model_selection import train_test_split
dataset_train,dataset_test = train_test_split(dataset, test_size=0.20, random_state=42)


In [ ]:
print(len(dataset_train))
print(len(dataset_test))

800
200


In [ ]:
print(dataset_train[0])

HeteroData(
  cRE={
    x=[100, 779],
    y=[10000]
  },
  gene={
    x=[100, 779],
    y=[10000]
  },
  (cRE, interactswith, cRE)={ edge_index=[2, 1869] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)


In [ ]:

from torch_geometric.nn.conv.hetero_conv import HeteroConv
from torch_geometric.nn.conv.gat_conv import GATConv
class network(torch.nn.Module):
    def __init__(self,gcnHiddensize,gcnOutputsize):
        super(network, self).__init__()
#         self.conv1 = GCNConv(gcnInputsize, gcnHiddensize)
        self.hetero_conv1 = HeteroConv({
            ('cRE','interactswith','cRE'): GCNConv(-1, gcnHiddensize, add_self_loops=False),
            ('cRE','regulates','gene'): GATConv((-1, -1), gcnHiddensize, add_self_loops=False),
        }, aggr='sum')
        self.hetero_conv2 = HeteroConv({
            ('cRE','interactswith','cRE'): GCNConv(-1, gcnHiddensize, add_self_loops=False),
            ('cRE','regulates','gene'): GATConv((-1, -1), gcnHiddensize, add_self_loops=False),
        }, aggr='mean')

        #self.conv2 = GCNConv(gcnHiddensize, gcnOutputsize)
        self.matrix=Variable(torch.randn(gcnHiddensize,gcnHiddensize).type(torch.FloatTensor),requires_grad=True)
        self.linear=torch.nn.Linear(1,2)
#         self.transform1=nn.Linear(indexsize, gcnInputsize)
#         self.transform2=nn.Linear(768,gcnInputsize)
#         self.indexsize=indexsize       
    def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x1=x[:,:self.indexsize].float()
#         x2=x[:,self.indexsize:].float()
#         x = self.transform1(x1)+self.transform2(x2)
#         print(data['cRE'])
        data_x = {'cRE':data['cRE'].x, 'gene': data['gene'].x}
        data_edge = {('cRE','interactswith','cRE'): data['cRE','interactswith','cRE'].edge_index, ('cRE','regulates','gene'): data['cRE','regulates','gene'].edge_index}
        x = self.hetero_conv1(data_x, data_edge)
        x = self.hetero_conv2(x, data_edge)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
        ypred=[]
        for i in range(x['cRE'].size(0)//100):
            cRE_batch = x['cRE'][i*100:(i+1)*100,:]
            gene_batch = x['gene'][i*100: (i+1) * 100, :]
            ypred.append(self.linear(torch.matmul(torch.matmul(cRE_batch,self.matrix),gene_batch.t()).unsqueeze(-1)).view(-1,2))
        
        return torch.stack(ypred).view(-1,2)
    
batchSize=16
feature=torch.tensor([10,9,8,7,6,5,4,3,2,1,0])

key_list=['A','B','C','D','E','F','G','H','I','J','K']
# key_list=['K','J','I','H','G','F','E','D','C','B','A']
dict_result={}
gcnInputsize=500
gcnHiddensize=300
gcnOutputsize=200
for i in range(11,12):
    featureIndex=torch.tensor(list(combinations(feature,r=i)))
    for index in tqdm(featureIndex):
        key="".join([key_list[i] for i in index])
        # print(key)
        # print(index)
        model=network(gcnHiddensize,gcnOutputsize)
        criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

     #   index1=torch.cat([index,torch.Tensor(list(range(11,779))).long()],dim=0)
        #dataset_train1 = [Data(x = torch.index_select(d.x, 1, index1), edge_index=d.edge_index, y=d.y) for d in dataset_train]
        loader = DataLoader(dataset_train, batch_size=batchSize, shuffle=True)

        for epoch in range(100):
            count=0
            loss=0
            for data1 in loader:
                pred_y = model(data1)
                loss = loss + criterion(pred_y, data1['gene'].y)
                print(data1['gene'].y)
                count=count+1
                if(count==10):
                    loss.backward()
                    optimizer.step()
                    print('epoch {}, loss {}'.format(epoch, loss.item()))
                    optimizer.zero_grad()
                    loss=0
                    count=0 
            if(loss!=0):
                loss.backward()
                optimizer.step()
                print('epoch {}, loss {}'.format(epoch, loss.item()))
                optimizer.zero_grad()


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
epoch 0, loss 55.37424850463867
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
epoch 0, loss 37.632965087890625
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])
tensor([-1, -1, -1,  ..., -1, -1, -1])

  0%|          | 0/1 [00:16<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
# index1=torch.cat([index,torch.Tensor(list(range(11,779))).long()],dim=0)                
# dataset_test1 = [Data(x = torch.index_select(d.x, 1, index1), edge_index=d.edge_index, y=d.y) for d in dataset_test]
testloader = DataLoader(dataset_test, batch_size=batchSize, shuffle=True)
from sklearn.metrics import f1_score
ytrue=[]
ypred=[]
with torch.no_grad():    
    for data in testloader:
        pred_y=model(data)
        v,i=torch.max(pred_y,dim=1)
        ytrue+=data['gene'].y.tolist()
        ypred+=i
ypred1 = [ypred[i] for i in range(len(ypred)) if ytrue[i] != -1] 
ytrue1 = [var for var in ytrue if var != -1] 
f1=f1_score(ytrue1,ypred1,average='macro')
print(f1)      
# dict_result.update({key:f1})
# out_file = open("dict_result_"+key+".json", "w")

# json.dump(dict_result, out_file, indent = 6)

# out_file.close()


0.9516120218579235


In [ ]:
truepos=0
falseneg=0
newpos=0
newneg=0
trueneg=0
falsepos=0
for i in range(len(ypred)):
    if(ytrue[i]==1):
        if(ypred[i]==1):
            truepos+=1
        else:
            falseneg+=1
    elif(ytrue[i]==-1):
        if(ypred[i]==1):
            newpos+=1
        else:
            newneg+=1
    elif(ytrue[i]==0):
        if(ypred[i]==1):
            falsepos+=1
        else:
            trueneg+=1
print(truepos,falseneg,newpos,newneg,falsepos,trueneg)



172 19 15665 1983852 3 289


In [ ]:
import copy
geneprediction=torch.zeros((cRE_att.size()[0]//100+1)*100,(GENE_att.size()[0]//100+1)*100)
# x_cpy = copy.deepcopy(x)
# x_cpy = torch.cat([x_cpy, torch.tensor(DNA)], 1)
# x_cpy = torch.cat([x_cpy, x_cpy[:10000 - x_cpy.size(0)]])
# print(x_cpy.size())
for i in tqdm(range(cRE_att.size()[0]//100+1)):
    for j in range(GENE_att.size()[0]//100+1):
        if((i+1)*100 > cRE_att.size()[0]):
            xcRE=list(range(i*100,cRE_att.size()[0])) + list(range(100-(cRE_att.size()[0]-(i*100))))
        else:
            xcRE=range(i*100,(i+1)*100)
        if((j+1)*100 > GENE_att.size()[0]):
            xgene=list(range(j*100,GENE_att.size()[0])) + list(range(100-(GENE_att.size()[0]-(j*100))))
        else:
            xgene=range(j*100,(j+1)*100)
        cREedgelista,cREedgelistb=(cRE_prediction[i*100:(i+1)*100,i*100:(i+1)*100] == 1).nonzero(as_tuple=True) 
        cREedgeindexfrom = cRE_edgeindex[0].tolist()
        cREedgeindexto = cRE_edgeindex[1].tolist()
        cREgoldpairs = list(zip(cREedgeindexfrom, cREedgeindexto))

        cREgoldsel = [(var[0] - i*100, var[1] - i*100) for var in cREgoldpairs 
                      if var[0] >= i*100 and var[1] >= i*100 and var[0] < (i+1)*100 and var[1] < (i+1)*100]

                      
        cREgoldsel += list(zip(cREedgelista.tolist(), cREedgelistb.tolist()))
        cREgoldsel = list(set(cREgoldsel))
        cREgoldselfrom = torch.LongTensor([var[0] for var in cREgoldsel])
        cREgoldselto = torch.LongTensor([var[1] for var in cREgoldsel])
        cREedgelista,cREedgelistb = (cREgoldselfrom, cREgoldselto)
        print('i and j are ',i,j)
        
        GENEedgelista,GENEedgelistb=(CGedge[i*100:(i+1)*100,j*100:(j+1)*100] == 1).nonzero(as_tuple=True)
        hetero2 = HeteroData()
        hetero2['cRE'].x= torch.index_select(cRE_all, 0, torch.tensor(xcRE).long())
        hetero2['gene'].x=torch.index_select(GENE_all, 0, torch.tensor(xgene).long())
#         hetero2['cRE'].y=torch.tensor(ycRE).long()
#         hetero2['gene'].y=torch.tensor(yGENE).long()
        hetero2['cRE','interactswith','cRE'].edge_index=torch.stack([cREedgelista,cREedgelistb]).long()
        hetero2['cRE','regulates','gene'].edge_index=torch.stack([GENEedgelista,GENEedgelistb]).long()
        print("hetero2 ",hetero2)
#         print(hetero2)
#         edge_index=[]
#         edgelista=[]
#         edgelistb=[]
#         x_dataset=x_cpy[i*100:(i+1)*100] + x_cpy[j*100:(j+1)*100]
#         edgelista,edgelistb=(isedge[i*100:(i+1)*100,j*100:(j+1)*100] == 1).nonzero(as_tuple=True)
#         edge_index = torch.stack([edgelista,edgelistb]).long()
#         datasetpred=Data(x=x_dataset, edge_index=torch.tensor(np.array(edge_index)).long())
        pred_y=model(hetero2)
        v,ind=torch.max(pred_y,dim=1)
        pred_edge_index = [var[0] - 1 for var in enumerate(ind,1) if var[1] == 1]
        tmp = torch.zeros(100,100)
        for point in pred_edge_index:
            point_x = point // 100
            point_y = point % 100
            tmp[point_x, point_y] = 1
        geneprediction[i*100:(i+1)*100, j*100:(j+1)*100] = tmp

  0%|          | 0/100 [00:00<?, ?it/s]

i and j are  0 0
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 16] }
)
i and j are  0 1
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  0 2
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  0 3
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  0 4
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  0 5
hetero2  HeteroData(
  cRE={ x=[

  1%|          | 1/100 [00:03<05:54,  3.58s/it]

i and j are  0 18
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  0 19
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5429] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  1 0
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5895] },
  (cRE, regulates, gene)={ edge_index=[2, 112] }
)
i and j are  1 1
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5895] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  1 2
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5895] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  1 3
hetero2  HeteroData(
  cRE={ 

  2%|▏         | 2/100 [00:06<04:54,  3.00s/it]

...

 28%|██▊       | 28/100 [01:25<03:32,  2.95s/it]

i and j are  27 19
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 6001] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  28 0
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5554] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  28 1
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5554] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  28 2
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5554] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  28 3
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5554] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)
i and j are  28 4
hetero2  HeteroData(
  cRE

 28%|██▊       | 28/100 [01:27<03:46,  3.14s/it]

i and j are  28 8
hetero2  HeteroData(
  cRE={ x=[100, 779] },
  gene={ x=[100, 779] },
  (cRE, interactswith, cRE)={ edge_index=[2, 5554] },
  (cRE, regulates, gene)={ edge_index=[2, 0] }
)


KeyboardInterrupt: ignored

In [ ]:
torch.save(geneprediction, 'genepreds_withknowenedge.pt')

In [ ]:
plt.imsave("geneknownplot.pdf",CGedge,cmap='bwr')

plt.imsave("geneplot-withknownedge.pdf",geneprediction,cmap='bwr')